In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/stroke_migration/LifeTrack_white_stork_clean.csv')

In [15]:
# get random sample of 1000 rows
df = df.sample(n=1000, random_state=1)
df = df[["timestamp", "location-long", "location-lat"]]
df.shape

(1000, 3)

In [16]:
# add coordinates in web mercator
from utils.geo import wgs84_to_web_mercator
df = wgs84_to_web_mercator(df, lon='location-long', lat='location-lat')

In [17]:
df

,timestamp,location-long,location-lat,x,y
321739,2015-04-22 17:15:06.000,24.952204,21.670569,2.777667e+06,2.472019e+06
81806,2013-08-16 12:05:07.000,15.605860,52.060456,1.737236e+06,6.811064e+06
424580,2015-05-25 18:45:07.000,25.929859,48.921427,2.886499e+06,6.261540e+06
165976,2014-08-20 18:34:04.000,25.722648,49.702630,2.863432e+06,6.394935e+06
127824,2014-03-13 10:15:07.000,25.726304,10.924742,2.863839e+06,1.223573e+06
...,...,...,...,...,...
683375,2015-08-12 15:10:07.000,24.686975,50.613579,2.748142e+06,6.553223e+06
64421,2014-05-04 13:15:08.000,34.682479,31.212205,3.860836e+06,3.660339e+06
481244,2014-09-04 16:10:06.000,27.112816,41.145275,3.018185e+06,5.033793e+06
571229,2015-04-12 07:25:06.000,37.671850,34.582185,4.193611e+06,4.107246e+06


In [18]:
# for this 1000 rows calculate distance to PW :)
from utils.geo import distance_wgs84, distance_web_mercator
# wgs84
PW_LONG = 21.010833
PW_LAT = 52.220404
# web mercator
PW_X = 2338915.2307025087
PW_Y = 6840075.854762697

df["distance_to_PW_wgs84"] = df.apply(lambda row: distance_wgs84(PW_LAT, PW_LONG, row["location-lat"], row["location-long"]), axis=1)
df["distance_to_PW_web_mercator"] = df.apply(lambda row: distance_web_mercator(PW_X, PW_Y, row["x"], row["y"]), axis=1)

In [19]:
df["distance_diff"] = df["distance_to_PW_wgs84"] - df["distance_to_PW_web_mercator"]

In [20]:
df

,timestamp,location-long,location-lat,x,y,distance_to_PW_wgs84,distance_to_PW_web_mercator,distance_diff
321739,2015-04-22 17:15:06.000,24.952204,21.670569,2.777667e+06,2.472019e+06,3.407494e+06,4.390037e+06,-9.825435e+05
81806,2013-08-16 12:05:07.000,15.605860,52.060456,1.737236e+06,6.811064e+06,3.703817e+05,6.023778e+05,-2.319961e+05
424580,2015-05-25 18:45:07.000,25.929859,48.921427,2.886499e+06,6.261540e+06,5.058494e+05,7.965876e+05,-2.907382e+05
165976,2014-08-20 18:34:04.000,25.722648,49.702630,2.863432e+06,6.394935e+06,4.334749e+05,6.879450e+05,-2.544701e+05
127824,2014-03-13 10:15:07.000,25.726304,10.924742,2.863839e+06,1.223573e+06,4.599479e+06,5.640979e+06,-1.041500e+06
...,...,...,...,...,...,...,...,...
683375,2015-08-12 15:10:07.000,24.686975,50.613579,2.748142e+06,6.553223e+06,3.119639e+05,4.997507e+05,-1.877868e+05
64421,2014-05-04 13:15:08.000,34.682479,31.212205,3.860836e+06,3.660339e+06,2.585869e+06,3.525191e+06,-9.393219e+05
481244,2014-09-04 16:10:06.000,27.112816,41.145275,3.018185e+06,5.033793e+06,1.315522e+06,1.929783e+06,-6.142608e+05
571229,2015-04-12 07:25:06.000,37.671850,34.582185,4.193611e+06,4.107246e+06,2.366922e+06,3.302765e+06,-9.358429e+05


In [21]:
df["distance_diff"].describe()

count    1.000000e+03
mean    -6.449028e+05
std      4.097528e+05
min     -1.151766e+06
25%     -1.032270e+06
50%     -7.421978e+05
75%     -1.626588e+05
max     -7.778779e+04
Name: distance_diff, dtype: float64

#### APPLY GRID ON WGS84 DATA

In [22]:
df = pd.read_csv('data/stroke_migration/LifeTrack_white_stork_clean.csv')

In [26]:
df["date"]

0         2013-07-16 12:04:53.000
1         2013-07-16 12:10:06.000
2         2013-07-16 12:15:07.000
3         2013-07-16 12:20:06.000
4         2013-07-16 12:25:07.000
                   ...           
751670    2015-07-19 15:10:51.000
751671    2015-07-19 15:21:08.000
751672    2015-07-19 15:21:08.000
751673    2015-07-19 15:42:29.000
751674    2015-07-19 15:42:29.000
Name: date, Length: 751675, dtype: object

In [23]:
df["location-lat"].describe()

count    751675.000000
mean         33.468379
std          18.590866
min         -26.925186
25%          14.585777
50%          39.074386
75%          52.738268
max          54.155769
Name: location-lat, dtype: float64

In [24]:
df["location-long"].describe()

count    751675.000000
mean         27.647333
std           7.044562
min          13.547600
25%          22.484694
50%          26.274487
75%          33.629192
max          44.437650
Name: location-long, dtype: float64